<a href="https://colab.research.google.com/github/syadegari/Libtorch_Colab/blob/main/scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install c++ highlighting and load it

In [ ]:
# Taken from: https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b
!wget -q -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py

In [ ]:
%load_ext cpp_plugin

The cpp_plugin extension is already loaded. To reload it, use:
  %reload_ext cpp_plugin


Download `libtorch` and unzip it

In [ ]:
%%bash

wget -q https://download.pytorch.org/libtorch/cu118/libtorch-cxx11-abi-shared-with-deps-2.0.1%2Bcu118.zip
unzip -qq libtorch-cxx11-abi-shared-with-deps-2.0.1+cu118.zip

In [ ]:
%%bash

mkdir -p src
mkdir -p build

In [ ]:
%%cpp -n src/main.cpp

#include <torch/torch.h>
#include <iostream>

int main() {
  std::cout << "PyTorch version: "
    << TORCH_VERSION_MAJOR << "."
    << TORCH_VERSION_MINOR << "."
    << TORCH_VERSION_PATCH << "\n";
}


In [ ]:
%%writefile CMakeLists.txt

# Specify the minimum required version of CMake
cmake_minimum_required(VERSION 3.12)

# Set the project name and version
project(MyProject VERSION 1.0)

# Define a variable for the program name
set(PROGRAM_NAME main)

set(CMAKE_VERBOSE_MAKEFILE ON)
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} ${TORCH_CXX_FLAGS}")
find_package(Torch REQUIRED PATHS /content/libtorch/share/cmake/Torch)

add_executable(${PROGRAM_NAME} src/main.cpp)
target_compile_options(${PROGRAM_NAME} PRIVATE -Wall -Wextra -Werror)
set_property(TARGET ${PROGRAM_NAME} PROPERTY CXX_STANDARD 20)
target_link_libraries(${PROGRAM_NAME} "${TORCH_LIBRARIES}")

# Diagnostic messages
message(STATUS "Compiler ID: ${CMAKE_CXX_COMPILER_ID}")
message(STATUS "Compiler Version: ${CMAKE_CXX_COMPILER_VERSION}")
message(STATUS "Compiler Path: ${CMAKE_CXX_COMPILER}")


Overwriting CMakeLists.txt


In [ ]:
%%cpp -n src/main.cpp

#include <torch/torch.h>
#include <iostream>

void foo(const auto& a, const auto& b) {
  std::cout << a << " " << b << "\n";
}


auto main() -> int {
  // Set device (use CUDA if available)
  torch::Device device =
      torch::cuda::is_available() ? torch::kCUDA : torch::kCPU;
  std::cout << "Detected device: "
            << (device.type() == torch::kCUDA ? "CUDA" : "CPU") << std::endl;
// testing c++20 auto-template
  foo(1, std::string("Hello!"));
  foo(std::string("Hello"), 42);
    return 0;
}

In [ ]:
%%bash

cd build
cmake --fresh ..
make
./main


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found CUDA: /usr/local/cuda (found version "11.8") 
-- The CUDA compiler identification is NVIDIA 11.8.89
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Caffe2: CUDA detected: 11.8
-- Caffe2: CUDA nvcc is: /usr/local/cuda/bin/nvcc
-- Caffe2: CUDA toolkit directory: /usr/local/cuda
-- Caffe2: Header version i

In [ ]:
ls build

CMakeCache.txt       detect_cuda_compute_capabilities.cu  Makefile
CMakeFiles/          detect_cuda_version.cc
cmake_install.cmake  main*
